## Advanced Box Score Calculations
Author: Akshay Easwaran <akeaswaran@me.com>

---
Based on work by ESPN's Bill Connelly, Football Outsiders, and Football Study Hall

Data from CollegeFootballData.com

Useful Resources for Stat Definitions:
- Football Study Hall website: https://www.footballstudyhall.com/
- Football Study Hall book: https://www.amazon.com/Study-Hall-College-Football-Stories/dp/1484989961
- Football Outsiders: https://www.footballoutsiders.com/info/glossary
- Example Advanced Box Score: https://twitter.com/ESPN_BillC/status/1176572006969597954/photo/1

**How to use this Notebook**

1. Dump a game's play by play data and drive data from CollegeFootballData.com.
2. Change the file names in the next cell to match where your data files are located.
3. Change the away and home team names appropriately.
4. Hit Cell > Run All Cells.

In [589]:
import pandas as pd

import requests
import pandas as pd
import json
import html
import os.path

selected_team = 'Georgia Tech'
selected_opponent = 'Clemson'
selected_week = 1
selected_team_is_home = False

def retrieveCfbData(endpoint, team, week):
    file_path = f"data/{endpoint if (endpoint != 'plays') else 'pbp'}/{endpoint[:-1] if (endpoint != 'plays') else 'pbp'}-data-{team.lower().replace(' ','-')}-wk{week}.json"
    if (os.path.exists(file_path)):
        return file_path
    res = requests.get(f"https://api.collegefootballdata.com/{endpoint}?seasonType=regular&year=2019&team={html.escape(team)}&week={week}")
    content = res.json()
    with open(file_path, 'w') as f:
        json.dump(content, f)
    return json.dumps(content)
    

pbp_data = pd.read_json(retrieveCfbData('plays',selected_team, selected_week))
# pbp_data.info()

drives = pd.read_json(retrieveCfbData('drives',selected_team, selected_week))
drives.drop(['offense_conference','start_time','end_time','defense_conference','elapsed','game_id'], axis = 1, inplace=True) 
drives = drives[
    (~drives.drive_result.isin(['Uncategorized']))
#     & (~drives.drive_result.str.contains('END OF'))
]

away_team = selected_team if ~selected_team_is_home else selected_opponent
away_score = 14
home_team = selected_opponent if ~selected_team_is_home else selected_team
home_score = 52

In [590]:
# Data Cleaning
# Fix the bad yard line markers for away teams
drives.loc[
    drives.offense == away_team, ['start_yardline']
] = 100 - drives.start_yardline
drives.loc[
    drives.offense == away_team, ['end_yardline']
] = 100 - drives.end_yardline

print("Total Drives:", len(drives))

pbp_data = pbp_data[
    (pbp_data.down != 0)
]

# Drop some unnecessary columns
pbp_data.drop(['offense_conference','defense_conference','clock'], axis = 1, inplace=True) 
# Ignore some types of plays cause they're special teams and weird
ignore_types = ["Defensive 2pt Conversion","Blocked Field Goal","Blocked Punt","Missed Field Goal Return","Blocked Punt Touchdown","Missed Field Goal Return Touchdown","Extra Point Missed","Extra Point Good","Timeout","End of Half","End of Game","Uncategorized","Penalty","Kickoff","Kickoff Return (Offense)","Kickoff Return Touchdown","Punt", "Field Goal Good","Field Goal Missed","Safety"]
pbp_data = pbp_data[~(pbp_data.play_type.isin(ignore_types))]
base_pbp_data = pbp_data.copy()

# Eliminate garbage time plays
garbage_time_data = pbp_data[
    (pbp_data.down == 0)
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 38) & (pbp_data.period == 2))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 28) & (pbp_data.period == 3))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 22) & (pbp_data.period == 4))
]

print("Total Plays:", len(pbp_data))
print("Garbage Time Plays:", len(garbage_time_data))
print("% of plays in garbage time:", len(garbage_time_data)/len(pbp_data))
pbp_data.drop(garbage_time_data.index, inplace=True)
print("Non-Garbage Time Plays:", len(pbp_data))

print("Total clean plays:", len(pbp_data))

Total Drives: 30
Total Plays: 142
Garbage Time Plays: 65
% of plays in garbage time: 0.45774647887323944
Non-Garbage Time Plays: 77
Total clean plays: 77


In [591]:
import numpy as np

pbp_data.distance = pbp_data.distance.astype(float)

bad_types = ["Interception","Pass Interception Return","Interception Return Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

pbp_data.loc[
    ((pbp_data.play_type.isin(bad_types))
     & (~pbp_data.play_type.str.contains('Sack'))) ,['yards_gained']] = 0

def is_successful(down, distance, yards_gained, play_type):
    if (play_type in bad_types):
        return False 
    if ((down == 1) & (yards_gained >= (0.5 * distance))):
        return True
    elif ((down == 2)) & (yards_gained >= (0.7 * distance)):
        return True
    elif ((down == 3) & (yards_gained >= distance)):
        return True
    elif ((down == 4) & (yards_gained >= distance)):
        return True
    else:
        return False
    
pbp_data['play_successful'] = np.vectorize(is_successful)(pbp_data.down, pbp_data.distance, pbp_data.yards_gained, pbp_data.play_type)
pbp_data.play_successful.value_counts()

base_pbp_data['play_successful'] = np.vectorize(is_successful)(base_pbp_data.down, base_pbp_data.distance, base_pbp_data.yards_gained, base_pbp_data.play_type)
base_pbp_data.play_successful.value_counts()

def is_explosive(yards_gained):
    if (yards_gained >= 15):
        return True
    else:
        return False
    
pbp_data['play_explosive'] = np.vectorize(is_explosive)(pbp_data.yards_gained)
base_pbp_data['play_explosive'] = np.vectorize(is_explosive)(base_pbp_data.yards_gained)

In [592]:
pass_types = ["Pass Reception","Pass Incompletion","Passing Touchdown","Interception","Pass Interception Return","Interception Return Touchdown","Sack"]
rush_types = ["Rush","Rushing Touchdown",'Fumble Recovery (Opponent)','Fumble Return Touchdown']

print("Success Rates")
print(away_team)
print("Overall:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))]))
print(home_team)
print("Overall:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))]))

print("")
print("Success Rate by Q")
for i in range(1, base_pbp_data.period.nunique()+1):
    print(f"Q{i}\n{away_team}:", len(base_pbp_data[(base_pbp_data.period == i) & (base_pbp_data.offense == away_team) & (base_pbp_data.play_successful == True)]) / len(base_pbp_data[(base_pbp_data.period == i) & (base_pbp_data.offense == away_team)]),f"\n{home_team}:", len(base_pbp_data[(base_pbp_data.period == i) & (base_pbp_data.offense == home_team) & (base_pbp_data.play_successful == True)]) / len(base_pbp_data[(base_pbp_data.period == i) & (base_pbp_data.offense == home_team)]))

Success Rates
Georgia Tech
Overall: 0.3684210526315789
Passing: 0.1
Rushing: 0.4642857142857143
Clemson
Overall: 0.46153846153846156
Passing: 0.3181818181818182
Rushing: 0.6470588235294118

Success Rate by Q
Q1
Georgia Tech: 0.26666666666666666 
Clemson: 0.5
Q2
Georgia Tech: 0.43478260869565216 
Clemson: 0.42105263157894735
Q3
Georgia Tech: 0.38461538461538464 
Clemson: 0.55
Q4
Georgia Tech: 0.4166666666666667 
Clemson: 0.45


In [593]:
###### Explosiveness rates
# Defined as rate of 15+ yards gains

print("Explosiveness Rates")
print(away_team)
print("Overall:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))]))
print(home_team)
print("Overall:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))]))

Explosiveness Rates
Georgia Tech
Overall: 0.02631578947368421
Passing: 0.0
Rushing: 0.03571428571428571
Clemson
Overall: 0.10256410256410256
Passing: 0.13636363636363635
Rushing: 0.058823529411764705


In [594]:
# Havoc plays
havoc_plays = base_pbp_data[
    (((base_pbp_data.play_type == 'Pass Incompletion')
    & (base_pbp_data.play_text.str.contains('broken up', regex=False)))
    | (base_pbp_data.play_type == 'Fumble Recovery (Opponent)')
    | (base_pbp_data.play_type == 'Sack')
    | (base_pbp_data.play_type.str.contains('Interception', regex=False))
    | (base_pbp_data.yards_gained < 0))
    & (base_pbp_data.play_type != 'Penalty')
]

print(away_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == away_team
]), "/", len(base_pbp_data[
    base_pbp_data.defense == away_team
]), "(",len(havoc_plays[
    base_pbp_data.defense == away_team
]) / len(base_pbp_data[
    base_pbp_data.defense == away_team
]),")")

print(home_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == home_team
]), "/", len(base_pbp_data[
    base_pbp_data.defense == home_team
]), "(",len(havoc_plays[
    havoc_plays.defense == home_team
]) / len(base_pbp_data[
    base_pbp_data.defense == home_team
]),")")

Georgia Tech Havoc Rate:  9 / 79 ( 0.11392405063291139 )
Clemson Havoc Rate:  11 / 63 ( 0.1746031746031746 )


/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [595]:
away_team_drives = drives[
    drives.offense == away_team
]

print(away_team, "Drives:", len(away_team_drives))
print(away_team,"Yards:",sum(away_team_drives.yards))
print(away_team,"Plays:",sum(away_team_drives.plays))
print(away_team, "Avg Starting Field Position:", sum(away_team_drives.start_yardline) / len(away_team_drives))
print(away_team, "Yards per Play:", sum(away_team_drives.yards) / sum(away_team_drives.plays))
print(away_team, "Plays per Drive:", sum(away_team_drives.plays) / len(away_team_drives))
print(away_team, "Yards per Drive:", sum(away_team_drives.yards) / len(away_team_drives))
print(away_team, "Points per Drive:", away_score / len(away_team_drives))
#away_team_drives
print("")
home_team_drives = drives[
    drives.offense == home_team
]

print(home_team, "Drives:", len(home_team_drives))
print(home_team,"Yards:",sum(home_team_drives.yards))
print(home_team,"Plays:",sum(home_team_drives.plays))
print(home_team, "Avg Starting Field Position:", sum(home_team_drives.start_yardline) / len(home_team_drives))
print(home_team, "Yards per Play:", sum(home_team_drives.yards) / sum(home_team_drives.plays))
print(home_team, "Plays per Drive:", sum(home_team_drives.plays) / len(home_team_drives))
print(home_team, "Yards per Drive:", sum(home_team_drives.yards) / len(home_team_drives))
print(home_team, "Points per Drive:", home_score / len(home_team_drives))
#home_team_drives

Georgia Tech Drives: 14
Georgia Tech Yards: 289
Georgia Tech Plays: 63
Georgia Tech Avg Starting Field Position: 28.285714285714285
Georgia Tech Yards per Play: 4.587301587301587
Georgia Tech Plays per Drive: 4.5
Georgia Tech Yards per Drive: 20.642857142857142
Georgia Tech Points per Drive: 1.0

Clemson Drives: 16
Clemson Yards: 594
Clemson Plays: 80
Clemson Avg Starting Field Position: 36.0
Clemson Yards per Play: 7.425
Clemson Plays per Drive: 5.0
Clemson Yards per Drive: 37.125
Clemson Points per Drive: 3.25


In [596]:
# Measuring success rate for a single player
# pbp_data[
#     (pbp_data.play_text.str.contains("Quentin Harris"))
#     & (pbp_data.play_type.isin(pass_types))
#     & (~pbp_data.play_type.str.contains("Sack"))
# ].play_successful.value_counts(normalize=True)

In [597]:
# Standard vs Passing Downs success rates
# Success rate on standard downs == leverage rate

standard_downs = pbp_data[
    (pbp_data.down == 1)
    | ((pbp_data.down == 2) & (pbp_data.distance <= 7))
    | ((pbp_data.down == 3) & (pbp_data.distance <= 4))
    | ((pbp_data.down == 4) & (pbp_data.distance <= 4)) 
]

passing_downs = pbp_data[
    ((pbp_data.down == 2) & (pbp_data.distance >= 8))
    | ((pbp_data.down == 3) & (pbp_data.distance >= 5))
    | ((pbp_data.down == 4) & (pbp_data.distance >= 5)) 
]

pass_plays = base_pbp_data[
    base_pbp_data.play_type.isin(pass_types)
]
rush_plays = base_pbp_data[
    base_pbp_data.play_type.isin(rush_types)
]

In [598]:
print(away_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == away_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == away_team)
]))

print(away_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == away_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == away_team)
]))

print(home_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == home_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == home_team)
]))

print(home_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == home_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == home_team)
]))

Georgia Tech Success Rate on Std Downs: 0.48
Georgia Tech Success Rate on Passing Downs: 0.15384615384615385
Clemson Success Rate on Std Downs: 0.48148148148148145
Clemson Success Rate on Passing Downs: 0.4166666666666667


In [599]:
# Stuff Rate
stuffed_plays = rush_plays[
    (rush_plays.yards_gained <= 0)
    & (rush_plays.play_type != 'Sack')
]

away_team_stuffs = stuffed_plays[
    stuffed_plays.defense == away_team
]
print(away_team,"Defensive Stuff Rate: ",len(away_team_stuffs),"/",len(rush_plays[rush_plays.defense == away_team]),"(",len(away_team_stuffs)/len(rush_plays[rush_plays.defense == away_team]),")")

home_team_stuffs = stuffed_plays[
    stuffed_plays.defense == home_team
]
print(home_team,"Defensive Stuff Rate: ",len(home_team_stuffs),"/",len(rush_plays[rush_plays.defense == home_team]),"(",len(home_team_stuffs)/len(rush_plays[rush_plays.defense == home_team]),")")

Georgia Tech Defensive Stuff Rate:  5 / 49 ( 0.10204081632653061 )
Clemson Defensive Stuff Rate:  10 / 42 ( 0.23809523809523808 )


In [600]:
# Opportunity Rate
rush_opps = rush_plays[
    (rush_plays.yards_gained >= 4)
]
print(away_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == away_team]),"/",len(rush_plays[rush_plays.offense == away_team]),"(",len(rush_opps[rush_opps.offense == away_team])/len(rush_plays[rush_plays.offense == away_team]),")")

print(home_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == home_team]),"/",len(rush_plays[rush_plays.offense == home_team]),"(",len(rush_opps[rush_opps.offense == home_team])/len(rush_plays[rush_plays.offense == home_team]),")")

Georgia Tech Rush Opp Rate:  19 / 42 ( 0.4523809523809524 )
Clemson Rush Opp Rate:  33 / 49 ( 0.673469387755102 )


In [601]:
##### Line Yards -- not adjusted for down/distance/opponent/shotgun
def adjust_strength_for_ol(yards_gained):
    if (yards_gained < 0):
        return yards_gained * 1.25
    elif ((yards_gained >= 0) & (yards_gained <= 3)):
        return yards_gained * 1.0
    elif ((yards_gained >= 4) & (yards_gained <= 6)):
        return yards_gained * 0.5
    else:
        return 0

rush_plays['line_yards'] = rush_plays.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_plays['highlight_yards'] = rush_plays.apply(lambda x: x.yards_gained - x.line_yards, axis=1)

rush_opps['line_yards'] = rush_opps.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_opps['highlight_yards'] = rush_opps.apply(lambda x: x.yards_gained - x.line_yards, axis=1)

rush_plays['line_yards'] = rush_plays.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_plays['highlight_yards'] = rush_plays.apply(lambda x: x.yards_gained - x.line_yards, axis=1)

rush_opps['line_yards'] = rush_opps.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_opps['highlight_yards'] = rush_opps.apply(lambda x: x.yards_gained - x.line_yards, axis=1)


print(away_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].line_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].highlight_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].yards_gained) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Hlt Yards per Opp:",sum(rush_opps[rush_opps.offense == away_team].highlight_yards) / len(rush_opps[rush_opps.offense == away_team]))

print("")
print(home_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].line_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].highlight_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].yards_gained) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Hlt Yards per Opp:",sum(rush_opps[rush_opps.offense == home_team].highlight_yards) / len(rush_opps[rush_opps.offense == home_team]))

rush_plays.head()

/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

Georgia Tech Line Yards per Carry: 0.3869047619047619
Georgia Tech Highlight Yards per Carry: 3.7797619047619047
Georgia Tech Yards per Carry: 4.166666666666667
Georgia Tech Hlt Yards per Opp: 8.026315789473685

Clemson Line Yards per Carry: 1.239795918367347
Clemson Highlight Yards per Carry: 7.1479591836734695
Clemson Yards per Carry: 8.387755102040817
Clemson Hlt Yards per Opp: 10.575757575757576


/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

,away,defense,defense_score,distance,down,drive_id,home,id,offense,offense_score,period,play_text,play_type,yard_line,yards_gained,play_successful,play_explosive,line_yards,highlight_yards
1,Georgia Tech,Georgia Tech,0,10,1,4011116531,Clemson,401111653101855872,Clemson,0,1,Travis Etienne run for a loss of 1 yard to the...,Rush,35,-1,False,False,-1.25,0.25
6,Georgia Tech,Georgia Tech,0,10,1,4011116532,Clemson,401111653101866880,Clemson,0,1,Travis Etienne run for a loss of 1 yard to the...,Rush,84,-1,False,False,-1.25,0.25
8,Georgia Tech,Georgia Tech,0,6,2,4011116532,Clemson,401111653101884672,Clemson,7,1,"Trevor Lawrence run for 6 yds for a TD, (B.T. ...",Rushing Touchdown,94,6,True,False,3.00,3.00
10,Georgia Tech,Clemson,7,10,1,4011116533,Clemson,401111653101885504,Georgia Tech,0,1,Tobias Oliver run for 2 yds to the GTech 27,Rush,75,2,False,False,2.00,0.00
11,Georgia Tech,Clemson,7,8,2,4011116533,Clemson,401111653101887872,Georgia Tech,0,1,Tobias Oliver run for 1 yd to the GTech 28,Rush,73,1,False,False,1.00,0.00


In [602]:
# Expected and Adjusted Turnover Margins

adj_turnover_plays = base_pbp_data[
    (base_pbp_data.play_type.str.contains('Interception', regex=False))
    | ((base_pbp_data.play_type == 'Pass Incompletion')
    & (base_pbp_data.play_text.str.contains('broken up', regex=False)))
    | (base_pbp_data.play_type.str.contains('Fumble', regex=False))
]

adj_fum = 0.49 * len(adj_turnover_plays[
    (adj_turnover_plays.play_type.str.contains('Fumble', regex=False))
])

# away_team Adj Turnovers
away_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == away_team)
    | (adj_turnover_plays.defense == away_team)
]

away_team_ints_def = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.defense == away_team)
])

away_team_ints_off = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.offense == away_team)
])

away_team_pds = len(away_team_tos[
   (away_team_tos.play_type == 'Pass Incompletion')
    & (away_team_tos.play_text.str.contains('broken up', regex=False))
    & (away_team_tos.offense == away_team)
])

away_team_fum_rec = away_team_tos[((away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.defense == away_team))]
away_team_fum_lost = away_team_tos[(away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.offense == away_team)]

In [603]:
# home_team Adj Turnovers (broken)
home_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == home_team)
    | (adj_turnover_plays.defense == home_team)
]

home_team_ints_def = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.defense == home_team)
])

home_team_ints_off = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.offense == home_team)
])

home_team_pds = len(home_team_tos[
   (home_team_tos.play_type == 'Pass Incompletion')
    & (home_team_tos.play_text.str.contains('broken up', regex=False))
    & (home_team_tos.offense == home_team)
])

home_team_fum_rec = home_team_tos[((home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.defense == home_team))]
home_team_fum_lost = home_team_tos[(home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.offense == home_team)]

print(away_team)
print("Def INTs:",away_team_ints_def)
print("Off INTs:",away_team_ints_off)
print("Def PDs:",(away_team_pds + away_team_ints_def))
print("Exp INTs:",0.22 * (away_team_pds + away_team_ints_off))
print("Fum Recovered:",len(away_team_fum_rec))
print("Fum Lost:",len(away_team_fum_lost))
print("Exp Fum:",adj_fum)
away_to = away_team_ints_off + len(away_team_fum_lost)
print("Actual TO:",away_to)
print("Actual TO Margin:",-1 * away_team_ints_off - len(away_team_fum_lost) + away_team_ints_def + len(away_team_fum_rec))
away_team_exp_to = 0.22 * (away_team_pds + away_team_ints_off) + adj_fum
print("Exp TO:",away_team_exp_to)

print("")

print(home_team)
print("Def INTs:",home_team_ints_def)
print("Off INTs:",home_team_ints_off)
print("Def PDs:",(home_team_pds + away_team_ints_def))
print("Exp INTs:",0.22 * (home_team_pds + home_team_ints_off))
print("Fum Recovered:",len(home_team_fum_rec))
print("Fum Lost:",len(home_team_fum_lost))
print("Exp Fum:",adj_fum)
home_to = home_team_ints_off + len(home_team_fum_lost)
print("Actual TO:",home_to)
print("Actual TO Margin:",-1 * home_team_ints_off - len(home_team_fum_lost) + home_team_ints_def + len(home_team_fum_rec))
home_team_exp_to = 0.22 * (home_team_pds + home_team_ints_off) + adj_fum
print("Exp TO:",home_team_exp_to)

Georgia Tech
Def INTs: 2
Off INTs: 2
Def PDs: 2
Exp INTs: 0.44
Fum Recovered: 1
Fum Lost: 1
Exp Fum: 0.98
Actual TO: 3
Actual TO Margin: 0
Exp TO: 1.42

Clemson
Def INTs: 2
Off INTs: 2
Def PDs: 2
Exp INTs: 0.44
Fum Recovered: 1
Fum Lost: 1
Exp Fum: 0.98
Actual TO: 3
Actual TO Margin: 0
Exp TO: 1.42


In [604]:
# home_team_tos

In [605]:
away_team_exp_to_margin = (home_team_exp_to - away_team_exp_to)
print("Exp TO Margin for",away_team,":", away_team_exp_to_margin)
home_team_exp_to_margin = (away_team_exp_to - home_team_exp_to)
print("Exp TO Margin for",home_team,":", home_team_exp_to_margin)

print("TO Luck (pts) for",away_team,":", ((home_to - away_to) - away_team_exp_to_margin) * 5.0)
print("TO Luck (pts) for",home_team,":", ((away_to - home_to) - home_team_exp_to_margin) * 5.0)

Exp TO Margin for Georgia Tech : 0.0
Exp TO Margin for Clemson : 0.0
TO Luck (pts) for Georgia Tech : 0.0
TO Luck (pts) for Clemson : 0.0


In [606]:
# Scoring Opportunities
# Definition: roughly, any time you get inside the opponent's 35, you should probably score

scoring_opps = drives[
    ((drives.start_yardline + drives.yards) >= 60)
]

# away team's scoring opps
print("Scoring Opportunities (IE: Drives inside Opponent's 40)")
print(away_team)
away_team_scoring_opps = scoring_opps[
    scoring_opps.offense == away_team
]
print("Total:",len(away_team_scoring_opps))
print("Scored:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]) / len(away_team_scoring_opps))
print("Opps/Drive:",len(away_team_scoring_opps) / len(drives[drives.offense == away_team]))
print("Points/Opp:",(away_score / len(away_team_scoring_opps)))

print("")
# home team's scoring opps
print(home_team)
home_team_scoring_opps = scoring_opps[
    scoring_opps.offense == home_team
]
print("Total:",len(home_team_scoring_opps))
print("Scored:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]) / len(home_team_scoring_opps))
print("Opps/Drive:",len(home_team_scoring_opps) / len(drives[drives.offense == home_team]))
print("Points/Opp:",(home_score / len(home_team_scoring_opps)))

Scoring Opportunities (IE: Drives inside Opponent's 40)
Georgia Tech
Total: 5
Scored: 2
Opp Efficiency: 0.4
Opps/Drive: 0.35714285714285715
Points/Opp: 2.8

Clemson
Total: 10
Scored: 8
Opp Efficiency: 0.8
Opps/Drive: 0.625
Points/Opp: 5.2


In [607]:
# Rushing Stats
rush_carries = rush_plays[
    (~rush_plays.play_type.isin(['Fumble Recovery (Opponent)','Fumble Return Touchdown']))
]

# look these up manually
away_team_rushers = ["Jordan Mason","Tobias Oliver","Jerry Howard Jr.","James Graham","Lucas Johnson","Jamious Griffin","Nathan Cottrell"]
home_team_rushers = ["Travis Etienne","Lyn-J Dixon","Chez Mellusi","Michel Dukes","Trevor Lawrence","Taisun Phommachanh","Darien Rencher","Chase Brice"]

def verify_division(num1, num2):
    if num2 == 0:
        return 0
    else:
        return num1 / num2

print("team level rushing stats")
print(away_team)
print(len(rush_carries[rush_carries.offense == away_team]),"car",
  sum(rush_carries[rush_carries.offense == away_team].yards_gained),"yards",
  len(rush_carries[(rush_carries.offense == away_team) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
  len(rush_plays[(rush_plays.offense == away_team) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
  "(",
  verify_division(sum(rush_carries[(rush_carries.offense == away_team)].yards_gained), len(rush_carries[(rush_carries.offense == away_team)])),"YPC",",",
  verify_division(sum(rush_carries[(rush_carries.offense == away_team)].line_yards), len(rush_carries[(rush_carries.offense == away_team)])),"line YPC",",",
  "SR%:",verify_division(len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))])),",",
  "Opp Rate:",verify_division(len(rush_opps[(rush_opps.offense == away_team)]), len(rush_carries[(rush_carries.offense == away_team)])),",",
  "Hlt/Opp:",verify_division(sum(rush_carries[(rush_carries.offense == away_team)].yards_gained), len(rush_opps[(rush_opps.offense == away_team)])),",",
  "Stuff%:",verify_division(len(stuffed_plays[stuffed_plays.offense == away_team]), len(pbp_data[pbp_data.offense == away_team])),
  ")")
print(home_team)
print(len(rush_carries[rush_carries.offense == home_team]),"car",
  sum(rush_carries[rush_carries.offense == home_team].yards_gained),"yards",
  len(rush_carries[(rush_carries.offense == home_team) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
  len(rush_plays[(rush_plays.offense == home_team) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
  "(",
  verify_division(sum(rush_carries[(rush_carries.offense == home_team)].yards_gained), len(rush_carries[(rush_carries.offense == home_team)])),"YPC",",",
  verify_division(sum(rush_carries[(rush_carries.offense == home_team)].line_yards), len(rush_carries[(rush_carries.offense == home_team)])),"line YPC",",",
  "SR%:",verify_division(len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))])),",",
  "Opp Rate:",verify_division(len(rush_opps[(rush_opps.offense == home_team)]), len(rush_carries[(rush_carries.offense == home_team)])),",",
  "Hlt/Opp:",verify_division(sum(rush_carries[(rush_carries.offense == home_team)].yards_gained), len(rush_opps[(rush_opps.offense == home_team)])),",",
  "Stuff%:",verify_division(len(stuffed_plays[stuffed_plays.offense == home_team]), len(pbp_data[pbp_data.offense == home_team])),
  ")")
    
print("")
print(away_team,"Rushing Stats")
for qb in away_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_type.isin(rush_types))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")

print("")

print(home_team,"Rushing Stats")
for qb in home_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_type.isin(rush_types))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")


team level rushing stats
Georgia Tech
41 car 179 yards 1 TDs 1 Fum ( 4.365853658536586 YPC , 0.5182926829268293 line YPC , SR%: 0.4642857142857143 , Opp Rate: 0.4634146341463415 , Hlt/Opp: 9.421052631578947 , Stuff%: 0.2631578947368421 )
Clemson
48 car 397 yards 5 TDs 1 Fum ( 8.270833333333334 YPC , 1.265625 line YPC , SR%: 0.6470588235294118 , Opp Rate: 0.6875 , Hlt/Opp: 12.030303030303031 , Stuff%: 0.1282051282051282 )

Georgia Tech Rushing Stats
Jordan Mason: 13 car 72 yards 1 TDs 0 Fum ( 5.538461538461538 YPC , 0.7692307692307693 line YPC , SR%: 0.6666666666666666 , Opp Rate: 0.6153846153846154 , Hlt/Opp: 9.0 )
Tobias Oliver: 18 car 74 yards 0 TDs 0 Fum ( 4.111111111111111 YPC , 0.09722222222222222 line YPC , SR%: 0.4 , Opp Rate: 0.3888888888888889 , Hlt/Opp: 10.571428571428571 )
Jerry Howard Jr.: 4 car 15 yards 0 TDs 0 Fum ( 3.75 YPC , 2.0 line YPC , SR%: 0.0 , Opp Rate: 0.25 , Hlt/Opp: 15.0 )
James Graham: 3 car 15 yards 0 TDs 0 Fum ( 5.0 YPC , 0.5833333333333334 line YPC , SR%: 

In [608]:
# Passing Stats
pass_attempts = pass_plays[
    (pass_plays.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Passing Touchdown')
    | (pass_plays.play_type == 'Pass Incompletion')
    | (pass_plays.play_type.str.contains('Interception'))
]

pass_completions = pass_attempts[
    (pass_attempts.play_type == 'Pass Reception')
    | (pass_attempts.play_type == 'Passing Touchdown')
]

for team in [away_team, home_team]:
    print(team + ":", 
      len(pass_completions[(pass_completions.offense == team)]),
      "/",len(pass_attempts[(pass_attempts.offense == team)]),
      sum(pass_completions[(pass_completions.offense == team)].yards_gained),"yards",
      len(pass_completions[(pass_completions.offense == team) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
      len(pass_attempts[(pass_attempts.offense == team) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
      len(base_pbp_data[(base_pbp_data.offense == team) & (base_pbp_data.play_type.str.contains("Sack"))]),"Sck")

print("")

# look these up manually
away_team_qbs = ["James Graham","Tobias Oliver","Lucas Johnson"]
home_team_qbs = ["Trevor Lawrence","Chase Brice"]

print(away_team,"Passing Stats")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(pass_plays[(pass_plays.play_text.str.contains(qb)) & (pass_plays.play_type.str.contains("Sack"))]),"Sck")
print("")
print(home_team,"Passing Stats")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(pass_plays[(pass_plays.play_text.str.contains(qb)) & (pass_plays.play_type.str.contains("Sack"))]),"Sck")

Georgia Tech: 7 / 18 137 yards 1 TDs 2 INTs 3 Sck
Clemson: 18 / 30 221 yards 2 TDs 2 INTs 0 Sck

Georgia Tech Passing Stats
James Graham: 4 / 7 72 yards 1 TDs 0 INTs 1 Sck
Tobias Oliver: 3 / 9 65 yards 0 TDs 2 INTs 2 Sck
Lucas Johnson: 0 / 2 0 yards 0 TDs 0 INTs 0 Sck

Clemson Passing Stats
Trevor Lawrence: 13 / 23 168 yards 1 TDs 2 INTs 0 Sck
Chase Brice: 5 / 7 53 yards 1 TDs 0 INTs 0 Sck


In [609]:
# Passing Stats on Std downs
pass_attempts = standard_downs[
    (standard_downs.play_type == 'Pass Reception')
    | (standard_downs.play_type == 'Passing Touchdown')
    | (standard_downs.play_type == 'Pass Incompletion')
    | (standard_downs.play_type.str.contains('Interception'))
]

pass_completions = standard_downs[
    (standard_downs.play_type == 'Pass Reception')
    | (standard_downs.play_type == 'Passing Touchdown')
]

print("Std Downs")

for team in [away_team, home_team]:
    print(team + ":", 
      len(pass_completions[(pass_completions.offense == team)]),
      "/",len(pass_attempts[(pass_attempts.offense == team)]),
      sum(pass_completions[(pass_completions.offense == team)].yards_gained),"yards",
      len(pass_completions[(pass_completions.offense == team) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
      len(pass_attempts[(pass_attempts.offense == team) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
      len(standard_downs[(standard_downs.offense == team) & (standard_downs.play_type.str.contains("Sack"))]),"Sck")

print("")

print(away_team,"Passing Stats on Std Downs")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(standard_downs[(standard_downs.play_text.str.contains(qb)) & (standard_downs.play_type.str.contains("Sack"))]),"Sck")
print("")
print(home_team,"Passing Stats on Std Downs")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(standard_downs[(standard_downs.play_text.str.contains(qb)) & (standard_downs.play_type.str.contains("Sack"))]),"Sck")

Std Downs
Georgia Tech: 1 / 5 0 yards 0 TDs 1 INTs 1 Sck
Clemson: 5 / 12 110 yards 1 TDs 1 INTs 0 Sck

Georgia Tech Passing Stats on Std Downs
James Graham: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck
Tobias Oliver: 1 / 4 0 yards 0 TDs 1 INTs 1 Sck
Lucas Johnson: 0 / 1 0 yards 0 TDs 0 INTs 0 Sck

Clemson Passing Stats on Std Downs
Trevor Lawrence: 5 / 12 110 yards 1 TDs 1 INTs 0 Sck
Chase Brice: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck


In [610]:
# Passing Stats on Pass. downs
pass_attempts = passing_downs[
    (passing_downs.play_type == 'Pass Reception')
    | (passing_downs.play_type == 'Passing Touchdown')
    | (passing_downs.play_type == 'Pass Incompletion')
    | (passing_downs.play_type.str.contains('Interception'))
]

pass_completions = passing_downs[
    (passing_downs.play_type == 'Pass Reception')
    | (passing_downs.play_type == 'Passing Touchdown')
]


print("Pass Downs")

for team in [away_team, home_team]:
    print(team + ":", 
      len(pass_completions[(pass_completions.offense == team)]),
      "/",len(pass_attempts[(pass_attempts.offense == team)]),
      sum(pass_completions[(pass_completions.offense == team)].yards_gained),"yards",
      len(pass_completions[(pass_completions.offense == team) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
      len(pass_attempts[(pass_attempts.offense == team) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
      len(passing_downs[(passing_downs.offense == team) & (passing_downs.play_type.str.contains("Sack"))]),"Sck")

print("")

print(away_team,"Passing Stats on Passing Downs")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
          len(passing_downs[(passing_downs.play_text.str.contains(qb)) & (passing_downs.play_type.str.contains("Sack"))]),"Sck")
print("")
print(home_team,"Passing Stats on Passing Downs")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(passing_downs[(passing_downs.play_text.str.contains(qb)) & (passing_downs.play_type.str.contains("Sack"))]),"Sck")

Pass Downs
Georgia Tech: 1 / 4 11 yards 0 TDs 0 INTs 0 Sck
Clemson: 7 / 10 38 yards 0 TDs 1 INTs 0 Sck

Georgia Tech Passing Stats on Passing Downs
James Graham: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck
Tobias Oliver: 1 / 3 11 yards 0 TDs 0 INTs 0 Sck
Lucas Johnson: 0 / 1 0 yards 0 TDs 0 INTs 0 Sck

Clemson Passing Stats on Passing Downs
Trevor Lawrence: 7 / 10 38 yards 0 TDs 1 INTs 0 Sck
Chase Brice: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck


In [611]:
# Recieving Stats
pass_attempts = base_pbp_data[
    (base_pbp_data.play_type == 'Pass Reception')
    | (base_pbp_data.play_type == 'Passing Touchdown')
    | (base_pbp_data.play_type == 'Pass Incompletion')
    | (base_pbp_data.play_type.str.contains('Interception'))
]

pass_completions = pass_attempts[
    (pass_attempts.play_type == 'Pass Reception')
    | (pass_attempts.play_type == 'Passing Touchdown')
]

print("Receiving  Stats")
# look these up manually
away_team_wrs = ["Jerry Howard Jr.","Josh Blancato","Ahmarean Brown","Malachi Carter","Tyler Davis","Jordan Mason"]
home_team_wrs = ["Tee Higgins","Lyn-J Dixon","Frank Ladson Jr.","Jaelyn Lay","Cornell Powell","Justyn Ross","Joe Ngata","J.C. Chalk","Will Swinney","Diondre Overton","Travis Etienne","Brannon Spector","T.J. Chase"]

print(away_team)
for wr in away_team_wrs:
    print(f"{wr}: {len(pass_attempts[pass_attempts.play_text.str.contains(wr)])} tgt, {len(pass_completions[pass_completions.play_text.str.contains(wr)])} catch, {sum(pass_completions[pass_completions.play_text.str.contains(wr)].yards_gained)} yds, SR%: {verify_division(len(pass_attempts[(pass_attempts.play_text.str.contains(wr) & (pass_attempts.play_successful == True))]),len(pass_attempts[pass_attempts.play_text.str.contains(wr)]))}")

print("")

print(home_team)   
for wr in home_team_wrs:
    print(f"{wr}: {len(pass_attempts[pass_attempts.play_text.str.contains(wr)])} tgt, {len(pass_completions[pass_completions.play_text.str.contains(wr)])} catch, {sum(pass_completions[pass_completions.play_text.str.contains(wr)].yards_gained)} yds, SR%: {verify_division(len(pass_attempts[(pass_attempts.play_text.str.contains(wr) & (pass_attempts.play_successful == True))]),len(pass_attempts[pass_attempts.play_text.str.contains(wr)]))}")

Receiving  Stats
Georgia Tech
Jerry Howard Jr.: 2 tgt, 2 catch, 57 yds, SR%: 0.5
Josh Blancato: 2 tgt, 1 catch, 34 yds, SR%: 0.5
Ahmarean Brown: 1 tgt, 1 catch, 28 yds, SR%: 1.0
Malachi Carter: 4 tgt, 1 catch, 11 yds, SR%: 0.25
Tyler Davis: 3 tgt, 1 catch, 7 yds, SR%: 0.3333333333333333
Jordan Mason: 2 tgt, 1 catch, 0 yds, SR%: 0.0

Clemson
Tee Higgins: 8 tgt, 4 catch, 98 yds, SR%: 0.5
Lyn-J Dixon: 1 tgt, 1 catch, 22 yds, SR%: 1.0
Frank Ladson Jr.: 1 tgt, 1 catch, 21 yds, SR%: 1.0
Jaelyn Lay: 1 tgt, 1 catch, 20 yds, SR%: 1.0
Cornell Powell: 2 tgt, 1 catch, 13 yds, SR%: 0.5
Justyn Ross: 6 tgt, 3 catch, 13 yds, SR%: 0.16666666666666666
Joe Ngata: 1 tgt, 1 catch, 12 yds, SR%: 1.0
J.C. Chalk: 1 tgt, 1 catch, 8 yds, SR%: 1.0
Will Swinney: 1 tgt, 1 catch, 4 yds, SR%: 0.0
Diondre Overton: 2 tgt, 1 catch, 3 yds, SR%: 0.0
Travis Etienne: 1 tgt, 1 catch, 3 yds, SR%: 0.0
Brannon Spector: 2 tgt, 1 catch, 3 yds, SR%: 0.0
T.J. Chase: 1 tgt, 1 catch, 1 yds, SR%: 0.0
